# Episode 1: Intro to Raster Data, Bad Data Values, Summary Stats

In [ ]:
import rasterio
import scipy.stats
import numpy
import earthpy.plot
import matplotlib.pyplot as plt

surface_model_HARV = rasterio.open("NEON-DS-Airborne-Remote-Sensing/HARV/DSM/HARV_dsmCrop.tif")
surface_model_HARV.meta

surface_model_HARV.meta['width']

surface_model_HARV_arr = surface_model_HARV.read(1)
surface_model_HARV.read(1)

earthpy.plot.plot_bands(
    surface_model_HARV_arr,
    scale=False,
    cmap="viridis",
    title="Digital Surface Model Without Hillshade",
    figsize=(10, 6)
)

plt.style.use("ggplot")
earthpy.plot.plot_bands(
     surface_model_HARV_arr,
     scale=False,
     cmap="viridis",
     title="Digital Surface Model Without Hillshade",
     figsize=(10, 6)
)

surface_model_HARV.crs

earthpy.epsg['32618']

scipy.stats.describe(surface_model_HARV_arr, axis=None)

print(numpy.percentile(surface_model_HARV_arr, 25))
print(numpy.percentile(surface_model_HARV_arr, 75))

surface_model_HARV.meta['count']

surface_model_HARV_arr_3D = surface_model_HARV.read()
surface_model_HARV_arr_3D.shape

surface_model_HARV_arr_2D = surface_model_HARV.read(1)
surface_model_HARV_arr_2D.shape

rgb_HARV = rasterio.open("NEON-DS-Airborne-Remote-Sensing/HARV/RGB_Imagery/HARV_RGB_Ortho.tif", "r")
rgb_HARV_arr = rgb_HARV.read()
earthpy.plot.plot_rgb(
    rgb_HARV_arr,
    title="RGB Image, NoData Values UnMasked",
    figsize=(10, 6)
)

rgb_HARV_masked_arr = numpy.ma.masked_where(rgb_HARV_arr==0, rgb_HARV_arr) #1st argument is the condition, second is the array to mask
earthpy.plot.plot_rgb(
    rgb_HARV_masked_arr,
    title="RGB Image, NoData Values Masked",
    figsize=(10, 6)
)

earthpy.plot.plot_rgb(
    rgb_HARV_masked_arr.mask*-1, # mutliplying a boolean array by -1 reverses True and False values
    title="Mask Array",
    figsize=(10, 6)
)

print(surface_model_HARV.nodata)
changed_meta_copy = surface_model_HARV.meta.copy()
changed_meta_copy['nodata'] = -3.4e+38
print(changed_meta_copy)


# Episode 2 Raster Reprojecting

In [ ]:
import os
os.chdir("../")

In [ ]:
import rasterio

surface_model_HARV = rasterio.open("data/NEON-DS-Airborne-Remote-Sensing/HARV/DSM/HARV_dsmCrop.tif")

terrain_model_HARV_WGS84 = rasterio.open("data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_WGS84.tif")

In [ ]:
surface_model_HARV.crs

In [ ]:
terrain_model_HARV_WGS84.crs

In [ ]:
import earthpy
earthpy.epsg["32618"]

In [ ]:
from rasterio.warp import calculate_default_transform, reproject

transform, width, height = calculate_default_transform(
    terrain_model_HARV_WGS84.crs, surface_model_HARV.crs, terrain_model_HARV_WGS84.width, terrain_model_HARV_WGS84.height, *terrain_model_HARV_WGS84.bounds)

destination_crs = surface_model_HARV.crs
reprojected_meta = terrain_model_HARV_WGS84.meta.copy()
reprojected_meta.update({
        'crs': destination_crs,
        'transform': transform,
        'width': width,
        'height': height
})

reprojected_path = "data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_UTM18_rasterio.tif"
with rasterio.open(reprojected_path, "w", **reprojected_meta) as reprojected_data:
    reproject(
        source = rasterio.band(terrain_model_HARV_WGS84, 1), 
        destination = rasterio.band(reprojected_data, 1), 
        src_crs=terrain_model_HARV_WGS84.crs, 
        dst_crs=surface_model_HARV.crs)

reprojected_tif = rasterio.open(reprojected_path)
terrain_model_HARV_arr_UTM18_rasterio = reprojected_tif.read()

In [ ]:
import xarray
import rioxarray

surface_model_HARV_xarr = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/HARV/DSM/HARV_dsmCrop.tif")
terrain_model_HARV_xarr = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_WGS84.tif")

surface_model_HARV_xarr

surface_model_HARV_xarr.rio.crs

terrain_model_HARV_xarr_UTM18 = terrain_model_HARV_xarr.rio.reproject(dst_crs=surface_model_HARV_xarr.rio.crs)

terrain_model_HARV_xarr_UTM18

surface_model_HARV_xarr.compute()

(terrain_model_HARV_xarr_UTM18 == terrain_model_HARV_arr_UTM18_rasterio).all()

reprojected_path = "data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_UTM18_rioxarray.tif"
terrain_model_HARV_xarr_UTM18.rio.to_raster(reprojected_path)

# view crs for DTM
print(terrain_model_HARV_xarr_UTM18.rio.crs)

# view crs for DSM
print(surface_model_HARV_xarr.rio.crs)

# view noddata value for DTM
print(terrain_model_HARV_xarr_UTM18.rio.nodata)

# view nodata value for DSM
print(surface_model_HARV_xarr.rio.nodata)

# view shape for DTM
print(terrain_model_HARV_xarr_UTM18.shape)

# view shape for DSM
print(surface_model_HARV_xarr.shape)

import matplotlib.pyplot as plt
terrain_model_HARV_xarr_UTM18.plot(cmap="viridis")
plt.title("Harvard Forest Digital Terrain Model")
plt.savefig("fig/02-bad-DTM-plot-01.png")

In [ ]:
surface_model_HARV_xarr.rio.crs

In [ ]:
terrain_model_HARV_xarr_UTM18.shape

In [ ]:
terrain_model_HARV_xarr.shape

In [ ]:
import matplotlib.pyplot as plt
terrain_model_HARV_xarr_UTM18_valid = terrain_model_HARV_xarr_UTM18.where(
    terrain_model_HARV_xarr_UTM18 != terrain_model_HARV_xarr_UTM18.rio.nodata)
terrain_model_HARV_xarr_UTM18_valid.plot(cmap="viridis")
plt.title("Harvard Forest Digital Terrain Model")
    

In [ ]:
import os
os.chdir("..")
import xarray
import rioxarray
import matplotlib.pyplot as plt
terrain_model_HARV_SJER = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/SJER/DTM/SJER_dtmCrop.tif")
surface_model_HARV_SJER = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/SJER/DSM/SJER_dsmCrop_WGS84.tif")
reprojected_surface_model = surface_model_HARV_SJER.rio.reproject(dst_crs=terrain_model_HARV_SJER.rio.crs)
reprojected_surface_model.rio.to_raster("data/NEON-DS-Airborne-Remote-Sensing/SJER/DSM/SJER_dsmCrop_WGS84.tif")
reprojected_surface_model.where(reprojected_surface_model != reprojected_surface_model.rio.nodata, drop=True).plot()
plt.title("SJER Reprojected Surface Model")
# plt.savefig("fig/02-SJER-DSM-03.png")

In [ ]:
terrain_model_HARV_SJER.where(terrain_model_HARV_SJER != terrain_model_HARV_SJER.rio.nodata).plot()
plt.title("SJER Terrain Model")
plt.savefig("fig/02-SJER-DTM-04.png")

In [ ]:
surface_model_HARV_SJER.shape

In [ ]:
rioxarray.__version__

In [ ]:
import os
os.chdir("..")
import xarray
import rioxarray

surface = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/HARV/DSM/HARV_dsmCrop.tif")
terrain = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop.tif")

terrain.where(terrain != terrain.rio.nodata).plot()

surface.plot()

print(terrain.shape)
print(surface.shape)

from rasterio.crs import CRS

terrain_WGS84 = terrain.rio.reproject(dst_crs=CRS.from_epsg(4326))
print(terrain_WGS84.shape)
terrain_WGS84.plot()

terrain_WGS84_to_UTM = terrain_WGS84.rio.reproject(dst_crs=terrain.rio.crs)

terrain_WGS84_to_UTM.where(terrain_WGS84_to_UTM!=terrain_WGS84_to_UTM.rio.nodata).plot()

(terrain_WGS84_to_UTM.sel(band=1)==terrain_WGS84_to_UTM.rio.nodata).plot() # a nodata border is created upon reprojecting back to UTM. this behavior makes it difficult to to do raster math with xarrays if you have to reproject any inputs

In [ ]:
mask

In [ ]:
import numpy as np
mask = np.all(terrain_WGS84_to_UTM.sel(band=1)==terrain_WGS84_to_UTM.rio.nodata, axis=1)
mask

In [ ]:
terrain_WGS84_to_UTM.shape

In [ ]:
terrain_WGS84.rio.to_raster("data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_WGS84.tif")

In [ ]:
terrain_WGS84_reread = xarray.open_rasterio("data/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_WGS84.tif")

In [ ]:
terrain_WGS84_reread.

In [ ]:
surface_model_HARV_xarr

surface_model_HARV_xarr.rio.crs

terrain_model_HARV_xarr_UTM18 = terrain_model_HARV_xarr.rio.reproject(dst_crs=surface_model_HARV_xarr.rio.crs)